In [37]:
!pip -q install ibm-watson-machine-learning
!pip -q install --upgrade ibm-watson-machine-learning

In [38]:
from ibm_watson_machine_learning import APIClient
from ibm_watson_machine_learning.metanames import AIServiceMetaNames

from pathlib import Path
import importlib.util, sys
import os, json, textwrap, datetime

API_KEY  = "zIyqNeDQ6Tkxilnpu-Sep0h4lX-kvEQP6-Z-LTZ5WpO4"     # <-- paste
SPACE_ID = "3dc35720-1a75-4e5f-a08e-f95f5eddb711"
URL      = "https://us-south.ml.cloud.ibm.com"

wml_credentials = {"url": URL, "apikey": API_KEY}
client = APIClient(wml_credentials)

# Set default space so store/deploy calls go to it
client.set.default_space(SPACE_ID)
print("Using space:", SPACE_ID)


ImportError: cannot import name 'AIServiceMetaNames' from 'ibm_watson_machine_learning.metanames' (/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/metanames.py)

In [17]:
import os, types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
import re
import base64, json, requests


def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rMxogpL4qsgtt0qT45To_N0q9iJp0gzili8mIWLk1RYr',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'riyassandbox-donotdelete-pr-spbpwn7zrlpz1g'
object_key = 'connectX.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   region_id                      500 non-null    object 
 1   region_name                    500 non-null    object 
 2   state                          500 non-null    object 
 3   population                     500 non-null    float64
 4   households                     500 non-null    float64
 5   population_density_km2         500 non-null    float64
 6   median_income_usd              500 non-null    float64
 7   education_index                500 non-null    float64
 8   urban_rural_flag               500 non-null    object 
 9   dsl_coverage_pct               500 non-null    float64
 10  avg_dsl_speed_mbps             500 non-null    float64
 11  competitor_fiber_presence      500 non-null    int64  
 12  complaint_rate_pct             500 non-null    flo

In [15]:
def score_take_rate(r):
    adoption = nz(r["broadband_adoption_pct"], 60) / 100
    income   = nz(r["median_income_usd"], 55000)
    income_norm = clamp((income - 30000) / 60000, 0, 1)
    comp_penalty      = 0.06 * nz(r["competitor_fiber_presence"], 0)
    complaint_penalty = nz(r["complaint_rate_pct"], 6) / 200
    urban_bonus = 0.05 if nz(r["urban_rural_flag"], "Rural") == "Urban" else 0
    take = 100 * clamp(0.25 + 0.5*adoption + 0.2*income_norm + urban_bonus
                       - comp_penalty - complaint_penalty, 0.05, 0.9)
    return round(take, 1)

def estimate_capex(r):
    density  = nz(r["population_density_km2"], 200)
    slope    = nz(r["terrain_slope_deg"], 3)
    backhaul = nz(r["existing_backhaul_distance_km"], 3)
    urban    = 1 if nz(r["urban_rural_flag"], "Rural") == "Urban" else 0
    capex = (1600
             + 220 * (1 / (1 + density/250))
             + 25 * slope
             + 90 * backhaul
             + 250 * (1 - urban))
    return round(capex, 2)

def predict_roi(r, take_rate, capex):
    arpu, opex_pct = 75.0, 0.35
    homes = max(1, int(nz(r["households"], 1)))
    subs = homes * (take_rate/100.0)
    annual_rev  = subs * arpu * 12
    annual_opex = annual_rev * opex_pct
    annual_cf   = max(1e-6, annual_rev - annual_opex)
    total_capex = homes * capex
    payback = total_capex / annual_cf
    roi3 = (3*annual_cf - total_capex) / max(1.0, total_capex) * 100
    return round(payback, 2), round(roi3, 1)

def priority_score(r):
    take = clamp(nz(r["take_rate_pct"], 0) / 80, 0, 1)
    roi  = clamp((nz(r["roi_3yr_pct"], -50) + 50) / 150, 0, 1)
    cap  = clamp(1 - (nz(r["capex_per_premise_usd"], 3200) - 1200) / 2000, 0, 1)
    comp = clamp(1 - nz(r["competitor_fiber_presence"], 0) / 3, 0, 1)
    ready = 0.6
    return round(0.30*take + 0.30*roi + 0.25*cap + 0.10*comp + 0.05*ready, 3)

def assign_phase_row(r):
    if r["priority_score"] >= 0.80 and r["payback_years"] <= 5.0 and r["capex_per_premise_usd"] <= 2000:
        return "Pilot"
    elif r["priority_score"] >= 0.65 and r["payback_years"] <= 5.0 and r["capex_per_premise_usd"] <= 2000:
        return "Phase 1"
    elif r["payback_years"] <= 6.5 and r["capex_per_premise_usd"] <= 2600:
        return "Phase 2"
    else:
        return "Defer"

def nz(x, default):
    return default if pd.isna(x) else x

def clamp(x, lo, hi): 
    try:
        x = float(x)
    except Exception:
        return lo
    return max(lo, min(hi, x))

In [26]:
code = """
import pandas as pd

def nz(x, default):
    return default if pd.isna(x) else x

def clamp(x, lo, hi): 
    try:
        x = float(x)
    except Exception:
        return lo
    return max(lo, min(hi, x))

def score_take_rate(r):
    adoption = nz(r.get("broadband_adoption_pct"), 60) / 100
    income   = nz(r.get("median_income_usd"), 55000)
    income_norm = clamp((income - 30000) / 60000, 0, 1)
    comp_penalty      = 0.06 * nz(r.get("competitor_fiber_presence"), 0)
    complaint_penalty = nz(r.get("complaint_rate_pct"), 6) / 200
    urban_bonus = 0.05 if nz(r.get("urban_rural_flag"), "Rural") == "Urban" else 0
    take = 100 * clamp(0.25 + 0.5*adoption + 0.2*income_norm + urban_bonus
                       - comp_penalty - complaint_penalty, 0.05, 0.9)
    return round(take, 1)

def estimate_capex(r):
    density  = nz(r.get("population_density_km2"), 200)
    slope    = nz(r.get("terrain_slope_deg"), 3)
    backhaul = nz(r.get("existing_backhaul_distance_km"), 3)
    urban    = 1 if nz(r.get("urban_rural_flag"), "Rural") == "Urban" else 0
    capex = (1600 + 220 * (1 / (1 + density/250)) + 25 * slope + 90 * backhaul + 250 * (1 - urban))
    return round(capex, 2)

def predict_roi(r, take_rate, capex):
    arpu, opex_pct = 75.0, 0.35
    homes = max(1, int(nz(r.get("households"), 1)))
    subs = homes * (take_rate/100.0)
    annual_rev  = subs * arpu * 12
    annual_opex = annual_rev * opex_pct
    annual_cf   = max(1e-6, annual_rev - annual_opex)
    total_capex = homes * capex
    payback = total_capex / annual_cf
    roi3 = (3*annual_cf - total_capex) / max(1.0, total_capex) * 100
    return round(payback, 2), round(roi3, 1)

def priority_score(r):
    take = clamp(nz(r.get("take_rate_pct"), 0) / 80, 0, 1)
    roi  = clamp((nz(r.get("roi_3yr_pct"), -50) + 50) / 150, 0, 1)
    cap  = clamp(1 - (nz(r.get("capex_per_premise_usd"), 3200) - 1200) / 2000, 0, 1)
    comp = clamp(1 - nz(r.get("competitor_fiber_presence"), 0) / 3, 0, 1)
    ready = 0.6
    return round(0.30*take + 0.30*roi + 0.25*cap + 0.10*comp + 0.05*ready, 3)

def assign_phase_row(r):
    if r["priority_score"] >= 0.80 and r["payback_years"] <= 5.0 and r["capex_per_premise_usd"] <= 2000:
        return "Pilot"
    elif r["priority_score"] >= 0.65 and r["payback_years"] <= 5.0 and r["capex_per_premise_usd"] <= 2000:
        return "Phase 1"
    elif r["payback_years"] <= 6.5 and r["capex_per_premise_usd"] <= 2600:
        return "Phase 2"
    else:
        return "Defer"

def connectx_service(context, params=None, **custom):
    # ONLINE inference entrypoint:
    def generate(context) -> dict:
        body = context.get_json() or {}
        path = body.get("path", "/priority")
        regions = body.get("regions", [])
        out = []

        for r in regions:
            if path == "/take-rate":
                out.append({"region_id": r.get("region_id"), "take_rate_pct": score_take_rate(r)})
            elif path == "/capex":
                out.append({"region_id": r.get("region_id"), "capex_per_premise_usd": estimate_capex(r)})
            elif path == "/roi":
                take = score_take_rate(r); capex = estimate_capex(r)
                payback, roi3 = predict_roi(r, take, capex)
                out.append({"region_id": r.get("region_id"),
                            "take_rate_pct": take, "capex_per_premise_usd": capex,
                            "payback_years": payback, "roi_3yr_pct": roi3})
            elif path == "/priority":
                take = score_take_rate(r); capex = estimate_capex(r)
                payback, roi3 = predict_roi(r, take, capex)
                rlocal = {"take_rate_pct": take, "capex_per_premise_usd": capex,
                          "payback_years": payback, "roi_3yr_pct": roi3,
                          "competitor_fiber_presence": r.get("competitor_fiber_presence")}
                out.append({"region_id": r.get("region_id"),
                            "priority_score": priority_score(rlocal)})
            elif path == "/phase":
                take = score_take_rate(r); capex = estimate_capex(r)
                payback, roi3 = predict_roi(r, take, capex)
                rlocal = {"take_rate_pct": take, "capex_per_premise_usd": capex,
                          "payback_years": payback, "roi_3yr_pct": roi3,
                          "competitor_fiber_presence": r.get("competitor_fiber_presence")}
                phase = assign_phase_row({"priority_score": priority_score(rlocal),
                                          "payback_years": payback,
                                          "capex_per_premise_usd": capex})
                out.append({"region_id": r.get("region_id"), "phase": phase})
            else:
                out.append({"region_id": r.get("region_id"), "error": f"Unknown path {path}"})

        return {"path": path, "results": out}
"""
with open("connectX_tools.py", "w") as f:
    f.write(code)
print("Wrote connectX_tools.py")

Wrote connectX_tools.py


In [27]:
import tarfile

with tarfile.open("fiber_function.tar.gz", "w:gz") as tar:
    tar.add("connectX_tools.py", arcname="connectX_tools.py")

print("Created fiber_function.tar.gz")

Created fiber_function.tar.gz


In [28]:
r = requests.post(
    "https://iam.cloud.ibm.com/identity/token",
    headers={"Content-Type":"application/x-www-form-urlencoded"},
    data={"grant_type":"urn:ibm:params:oauth:grant-type:apikey","apikey":API_KEY}
)
token = r.json()["access_token"]
print("Got IAM token ✔")

Got IAM token ✔


In [18]:
with open("function.tar.gz", "rb") as f:
    b64 = base64.b64encode(f.read()).decode("utf-8")

In [24]:
SPACE_ID  = "3dc35720-1a75-4e5f-a08e-f95f5eddb711"         # from your Developer access card
PROJECT_ID= "08ce3da9-7da1-440b-9dc0-d6654da40a43"                            # from the same card (shown in your screenshot)
URL       = "https://us-south.ml.cloud.ibm.com"
VERSION   = "2024-08-01"  

In [35]:
client = APIClient({"url": URL, "apikey": API_KEY})
client.set.default_space(SPACE_ID)
client.set.default_project(PROJECT_ID)

# 2.1 Load the function symbol from ai_service.py
spec = importlib.util.spec_from_file_location("ai_mod", "connectX_tools.py")
ai_mod = importlib.util.module_from_spec(spec)
sys.modules["ai_mod"] = ai_mod
spec.loader.exec_module(ai_mod)

SSPEC_ID = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")  # or another shown in the list


Unsetting the space_id ...


In [36]:
meta = {
    client.repository.ModelMetaNames.NAME: "connectX-tools",
    client.repository.ModelMetaNames.TYPE: "function",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
}
model_id = client.repository.store_model(model="connectX_tools.py", meta_props=meta)
print("MODEL_ID:", model_id)


AttributeError: 'Repository' object has no attribute 'AIServiceMetaNames'

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient({"url": "https://us-south.ml.cloud.ibm.com", "apikey": "zIyqNeDQ6Tkxilnpu-Sep0h4lX-kvEQP6-Z-LTZ5WpO4"})
client.set.default_space("3dc35720-1a75-4e5f-a08e-f95f5eddb711")

meta = {
  client.repository.ModelMetaNames.NAME: "connectX-tools",
  client.repository.ModelMetaNames.TYPE: "function",
  client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
}

# model_id = client.repository.store_model(model="connectX_tools.py", meta_props=meta)
# deployment = client.deployments.create(artifact_uid=model_id, meta_props={client.deployments.ConfigurationMetaNames.NAME: "connectX-tools-api"})
# print(client.deployments.get_scoring_href(deployment))

model_id = client.repository.store_model(model="connectX_tools.py", meta_props=meta)

dep = client.deployments.create(
    artifact_uid=model_id,
    meta_props={client.deployments.ConfigurationMetaNames.NAME: "connectX-tools-api"}
)
deployment_id = client.deployments.get_id(dep)
scoring_url = client.deployments.get_scoring_href(dep)
deployment_id, scoring_url


In [ ]:
df["take_rate_pct"] = df.apply(score_take_rate, axis=1)
df["capex_per_premise_usd"] = df.apply(estimate_capex, axis=1)
df[["payback_years","roi_3yr_pct"]] = df.apply(
    lambda r: pd.Series(predict_roi(r, r["take_rate_pct"], r["capex_per_premise_usd"])),
    axis=1
)
df["priority_score"] = df.apply(priority_score, axis=1)
df = df.sort_values("priority_score", ascending=False).reset_index(drop=True)
df["priority_rank"] = df.index + 1
df["phase"] = df.apply(assign_phase_row, axis=1)
df.head(10)

In [ ]:
df.describe()[["take_rate_pct","capex_per_premise_usd","payback_years","roi_3yr_pct","priority_score"]]

In [ ]:
connectX_output_data = df.to_csv("connectX_data_output.csv",index=False)
try:
    cos_client.upload_file(Filename="connectX_output_data", Bucket=bucket, Key=object_key)
    print(f"File '{file_name}' uploaded to '{object_key}' in bucket '{bucket_name}'.")
except Exception as e:
    print(f"Error uploading file: {e}")
    
